## Non-massaction propensities in BioCRNpyler
The propensity $\rho(s)$ (or rate function) of a chemical reaction determines how quickly that reaction occurs. It is assumed to be a function of the chemical species $s$ as well as internal parameter specific to the propensity type.

\begin{equation}
\{\textrm{Input Species}\} \xrightarrow{\rho(s)} \{\textrm{Output Species}\}
\end{equation}

By default, BioCRNpyler assumes that propensities are massaction with only one parameter, the rate constant $k$. However, non-massaction propensities are also supported. When creating a reaction with non-massaction propensities, a propensity_params dictionary must be passed into the chemical_reaction_network.reaction constructor with the appropriate parameters for that particular propensity type. The supported propensity types and their relevant parameters are:

1. "massaction: $\rho(S) = k \Pi_{s} s^{I_s}$. Required parameters: "k" the rate constant. Note: for stochastic models mass action propensities are $\rho(S) = k \Pi_{s} s!/(s - I_s)!$.
2. "positivehill": $\rho(s) = k \frac{s^n}{(K^n+s^n)}$. Requried parameters: rate constant "k", offset "K", hill coefficient "n", hill species "s1".
3. "negativehill": $\rho(s) = k \frac{1}{(K^n+s^n)}$. Requried parameters: rate constant "k", offset "K", hill coefficient "n", hill species "s1".
4. "proportionalpositivehill": $\rho(s) = k d \frac{s^n}{(K^n+s^n)}$. Requried parameters: rate constant "k", offset "K", hill coefficient "n", hill species "s1", propritional species "d".
5. "proportionalnegativehill": $\rho(s) = k d \frac{1}{(K^n+s^n)}$. Requried parameters: rate constant "k", offset "K", hill coefficient "n", hill species "s1", propritional species "d".
6. "general": $\rho(s) = f(s)$ where $f$ can be any algebraic function typed as a string. Required parameters: "rate" an algebraic expression including species and model parameters written as a string.

The following notebook contains examples of the propensity types support by BioCRNpyler.

In [4]:
from biocrnpyler.chemical_reaction_network import Species, Reaction, ComplexSpecies, ChemicalReactionNetwork


#Names of different supported propensities
propensity_types = ['hillpositive', 'proportionalhillpositive', 'hillnegative', 'proportionalhillnegative', 'massaction', 'general']

kb = 100
ku = 10
kex = 1.

G = Species(name = "G", material_type = "dna") #DNA
A = Species(name = "A", material_type = "protein") #Activator
R = Species(name = "A", material_type = "protein") #Repressor
GA = ComplexSpecies([G, A, A]) #Activated Gene
GR = ComplexSpecies([G, R, R]) #Repressed Gene
X = Species(name = "X", material_type = "protein")


#Massaction Unregulated
species1 = [G, A, R, GA, GR, X]
rxn1_1 = Reaction([G, A, A], [GA], k=kb, k_rev = ku)
rxn1_2 = Reaction([G, R, A], [GR], k=kb, k_rev = ku)
rxn1_3 = Reaction([GA], [GA, X], k=kex)
CRN1 = ChemicalReactionNetwork(species1, [rxn1_1, rxn1_2, rxn1_3])

#hill positive
species2 = [G, A, X]
rxn2_1 = Reaction([G], [G, X], propensity_type = "hillpositive", propensity_params = {"k":kex, "n":2, "K":float(kb/ku), "s1":A})
CRN2 = ChemicalReactionNetwork(species2, [rxn2_1])

#proportional hill positive
species3 = [G, A, X]
rxn3_1 = Reaction([G], [G, X], propensity_type = "proportionalhillpositive", propensity_params = {"k":kex, "n":2, "K":float(kb/ku), "s1":A, "d":G})
CRN3 = ChemicalReactionNetwork(species3, [rxn3_1])

#hill Negative
species4 = [G, R, X]
rxn4_1 = Reaction([G], [G, X], propensity_type = "hillnegative", propensity_params = {"k":kex, "n":2, "K":float(kb/ku), "s1":R})
CRN4 = ChemicalReactionNetwork(species4, [rxn3_1])

#proportional hill negative
species5 = [G, R, X]
rxn5_1 = Reaction([G], [G, X], propensity_type = "proportionalhillnegative", propensity_params = {"k":kex, "n":2, "K":float(kb/ku), "s1":A, "d":G})
CRN5 = ChemicalReactionNetwork(species5, [rxn3_1])


fname = "CRN.xml"

CRN = CRN5

import numpy as np
x0 = {"G":1, "A":10, "R":10}
timepoints = np.linspace(0, 10, 100)
CRN.write_sbml_file(file_name = fname)
#CRN.simulate_with_bioscrape(timepoints, initial_condition_dict = x0)
R = CRN.simulate_with_bioscrape_via_sbml(timepoints, initial_condition_dict = x0)

C:\ProgramData\Anaconda3\lib\site-packages\biocrnpyler-0.1-py3.7.egg\biocrnpyler\chemical_reaction_network.py:439: UserWarning: Reaction dna_G + 2*protein_A <--> complex_protein_A_protein_A_dna_G        massaction: k_f=100,	k_r=10 duplicated in CRN definitions. Duplicates have been removed.
  warn(f"Reaction {r} duplicated in CRN definitions. Duplicates "
C:\ProgramData\Anaconda3\lib\site-packages\bioscrape\sbmlutil.py:184: UserWarning: Compartments, UnitDefintions, Events, and some other SBML model components are not recognized by bioscrape. Refer to the bioscrape wiki for more information.
  warnings.warn('Compartments, UnitDefintions, Events, and some other SBML model components are not recognized by bioscrape. ' +
C:\ProgramData\Anaconda3\lib\site-packages\biocrnpyler-0.1-py3.7.egg\biocrnpyler\chemical_reaction_network.py:631: UserWarning: Trying to set species that is not in model: G
  m.set_species(initial_condition_dict)
C:\ProgramData\Anaconda3\lib\site-packages\biocrnpyler-0.1